In [1]:
import numpy as np, pandas as pd
from scipy.sparse import coo_matrix
from implicit.bpr import BayesianPersonalizedRanking
import recometrics
from poismf import PoisMF
from pandas.api.types import CategoricalDtype
from scipy import sparse
from sklearn.model_selection import train_test_split
import recmetrics
import tqdm


In [2]:
data = pd.read_csv("ml100k_ratings.csv",sep=",")


In [3]:
train_df, test_df = train_test_split(data, test_size=0.3, random_state=42)


In [4]:
user_positives = train_df.groupby('userId')['movieId'].apply(set).to_dict()


In [5]:
# from https://www.geeksforgeeks.org/python-pandas-dataframe-where/
users = train_df['userId'].unique()
items = train_df['movieId'].unique()
users = sorted(users)
items = sorted(items)

# HW code
ratingMatrix = test_df.pivot(index='userId', columns='movieId', values='rating')
ratingMatrix = ratingMatrix.reindex(index=users, columns=items)
ratingMatrix.fillna(0)

display(ratingMatrix)

movieId,1,2,3,4,5,6,7,8,9,10,...,1667,1668,1670,1671,1672,1673,1676,1678,1679,1680
userId,,,,,,,,,,,,,,,,,,,,,
1,5.0,NaN,4.0,3.0,NaN,5.0,NaN,1.0,NaN,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
940,NaN,NaN,NaN,NaN,NaN,NaN,4.0,5.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
941,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
# From https://hippocampus-garden.com/pandas_sparse/
users = train_df["userId"].unique()
movies = train_df["movieId"].unique()
shape = (len(users), len(movies))

# Create indices for users and movies
user_cat = CategoricalDtype(categories=sorted(users), ordered=True)
movie_cat = CategoricalDtype(categories=sorted(movies), ordered=True)
user_index = train_df["userId"].astype(user_cat).cat.codes
movie_index = train_df["movieId"].astype(movie_cat).cat.codes

# Conversion via COO matrix
coo = sparse.coo_matrix((train_df["rating"], (user_index, movie_index)), shape=shape)
csr = coo.tocsr()


In [7]:
# From https://hippocampus-garden.com/pandas_sparse/
bpr = BayesianPersonalizedRanking(factors=19, learning_rate=0.001,
                                  regularization=0.001, dtype=np.float64,
                                  iterations=100)

In [8]:
# From https://hippocampus-garden.com/pandas_sparse/
bpr.fit(csr)


  0%|          | 0/100 [00:00<?, ?it/s]

In [9]:
# compute dot product for results
pred = np.matmul(bpr.user_factors,bpr.item_factors.T)

In [10]:
from recsys_metrics import *
import torch
# https://github.com/zuoxingdong/recsys_metrics?tab=readme-ov-file#Citation

In [11]:
ratingMatrix[ratingMatrix >1] = 1
true = torch.tensor(ratingMatrix.fillna(0).to_numpy())
# true result needs to be be binary values for metric evaluation

In [12]:
predv = torch.tensor(pred)
# https://github.com/zuoxingdong/recsys_metrics?tab=readme-ov-file#Citation

In [13]:
display(true)
display(predv)

tensor([[1., 0., 1.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]], dtype=torch.float64)

tensor([[ 4.6950e-01,  3.1426e-02, -4.7893e-02,  ..., -2.1799e-02,
          1.3673e-03,  1.5826e-02],
        [ 4.7889e-01,  2.8286e-02, -4.5015e-02,  ..., -2.1261e-02,
         -1.7491e-03,  1.5230e-02],
        [ 4.7925e-01,  2.3089e-02, -4.4705e-02,  ..., -2.0683e-02,
         -4.5912e-04,  1.3327e-02],
        ...,
        [ 4.7582e-01,  2.6850e-02, -4.4378e-02,  ..., -2.1117e-02,
         -3.3919e-03,  1.3620e-02],
        [ 4.7849e-01,  2.7471e-02, -4.7511e-02,  ..., -1.9580e-02,
          1.2944e-03,  1.6058e-02],
        [ 4.7760e-01,  2.9556e-02, -4.3486e-02,  ..., -2.2697e-02,
         -2.5169e-03,  1.3014e-02]], dtype=torch.float64)

In [17]:
# Jinming code
def calculate_metrics(pred_df, user_postivies, k_list = [10,50,100]):
  metrics = {
    'MAR@ALL': [],
    'MAR@1000': [],
  }


  grouped = pred_df.groupby('user')

  for user, user_data in tqdm.tqdm(grouped, desc="Processing Users"):
    positives = user_postivies.get(user, set())
    if len(positives) == 0:
      continue

    sorted_data = user_data.sort_values(by='score', ascending=False)
    all_items = sorted_data['item'].tolist()

    relevant_ranks1k = []
    relevant_ranks_all=[]
    for rank, item in enumerate(all_items[:1000], start=1):
      if item in positives:
        relevant_ranks1k.append(rank)
    #calculate relevant
    for rank, item in enumerate(all_items, start=1):
      if item in positives:
        relevant_ranks_all.append(rank)



    if relevant_ranks_all:
      metrics['MAR@ALL'].append(np.mean(relevant_ranks_all))
    if relevant_ranks1k:
      metrics['MAR@1000'].append(np.mean(relevant_ranks1k))
      

  
  final = {
    'MAR@ALL': np.mean(metrics['MAR@ALL']),
    'MAR@1000': np.mean(metrics['MAR@1000']),
  }

  return final

In [18]:
# Toss https://github.com/zuoxingdong/recsys_metrics?tab=readme-ov-file#Citation
# functions here as a group
def metricStuff(pred,true,user_positive):
    print("NDCG@10",normalized_dcg(pred, true ,k=10))
    print("MRR@1000",mean_reciprocal_rank(pred, true,k=1000))
    print("MRR@all",mean_reciprocal_rank(pred, true))
    # From https://www.geeksforgeeks.org/how-to-convert-torch-tensor-to-pandas-dataframe/
    marData = pred.detach().numpy()
    marDF = pd.DataFrame(marData)
    marDF.index.name = "user"
    marDF.columns.name = "items"
    # From https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.stack.html
    # From https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.reset_index.html
    marDF = marDF.stack().reset_index()
    marDF.columns = ['user', 'item', 'score']
    print("MAR",calculate_metrics(marDF,user_positive))
    print("MAP",mean_average_precision(pred, true))
    print("P@50",precision(pred, true,k=50))
    print("R@50",recall(pred, true,k=50))

In [19]:
metricStuff(predv,true,user_positives)

NDCG@10 tensor(0.1593)
MRR@1000 tensor(0.3572)
MRR@all tensor(0.3572)


Processing Users: 100%|██████████| 943/943 [00:00<00:00, 1103.28it/s]


MAR {'MAR@ALL': np.float64(580.9361540120927), 'MAR@1000': np.float64(335.28260517270417)}
MAP tensor(0.1020)
P@50 tensor(0.1105)
R@50 tensor(0.2097)
